# Geospatial Data Assessment - DRC

In [1]:
import sys, os
import rasterio, geojson, folium, shapely

import pandas as pd
import geopandas as gpd
import numpy as np

from shapely import geometry, wkt
from folium import plugins

sys.path.insert(0, "/home/wb411133/Code/gostrocks/src")
import GOSTRocks.rasterMisc as rMisc
from GOSTRocks.misc import tPrint

rMisc.clipRaster?

In [2]:
# define input data
input_folder = "/home/wb411133/data/Projects/FDP_Geospatial/AOI/DRC"
health_zones = os.path.join(input_folder, "HealthZones", "RDC_Zones de santé.shp")
sel_zones = os.path.join(input_folder, "HealthZones", "selectionHZ.csv")

buildings_folder = "/home/public/Data/GLOBAL/Buildings/"
global_urban = "/home/public/Data/GLOBAL/URBAN/GHS/GHS_SMOD/GHS_SMOD_POP2015_GLOBE_R2019A_54009_1K_V2_0.tif"
global_pop   = "/home/public/Data/GLOBAL/Population/GHS/GHS_POP_E2015_GLOBE_R2019A_54009_1K_V1_0.tif"

# define local files
local_urban = os.path.join(input_folder, "GHS_SMOD.tif")
local_pop = os.path.join(input_folder, "GHS_POP.tif")

In [3]:
inD = gpd.read_file(health_zones)
selD = pd.read_csv(sel_zones)
selD = selD.loc[selD['Selected'] == 1]

In [4]:
selD = selD.merge(inD, left_on='Health_Zone', right_on="Nom", how='left')
selD = gpd.GeoDataFrame(selD, geometry='geometry', crs=inD.crs)

In [ ]:
selD.to_file(os.path.join(input_folder, "health_zones_selected.geojson"), driver="GeoJSON")

In [5]:
# map selected zones
m = folium.Map(location=[selD.unary_union.centroid.y,selD.unary_union.centroid.x], zoom_start=11)

for idx, row in selD.iterrows():
    geo_h = folium.GeoJson(shapely.geometry.mapping(row['geometry']), 
                         style_function=lambda x: {'fillColor': 'red'})
    geo_h.add_to(m)

m

# Extract secondary data

In [6]:
# Identify google footprints tiles
buildings_footprints = "/home/public/Data/GLOBAL/Buildings/google_tiles.geojson"
inF = gpd.read_file(buildings_footprints)
inF.head()

,tile_id,tile_url,size_mb,geometry
0,025,https://storage.googleapis.com/open-buildings-...,0.2,"POLYGON ((-16.53484 -38.79460, -10.61966 -39.4..."
1,04f,https://storage.googleapis.com/open-buildings-...,0.7,"POLYGON ((-10.61966 -16.26673, -5.05987 -16.47..."
2,05b,https://storage.googleapis.com/open-buildings-...,0.2,"POLYGON ((-16.53484 -10.18989, -10.61966 -10.4..."
3,093,https://storage.googleapis.com/open-buildings-...,21.0,"POLYGON ((-28.67315 9.34189, -22.61986 9.81930..."
4,095,https://storage.googleapis.com/open-buildings-...,14.2,"POLYGON ((-28.67315 14.59941, -22.61986 15.325..."


In [7]:
#Download intersecting buildings manually
selF = inF.loc[inF.intersects(selD.unary_union)]
sel_tiles = selF['tile_url'].values
sel_tiles

array(['https://storage.googleapis.com/open-buildings-data/v1/polygons_s2_level_4_gzip/199_buildings.csv.gz',
       'https://storage.googleapis.com/open-buildings-data/v1/polygons_s2_level_4_gzip/19f_buildings.csv.gz',
       'https://storage.googleapis.com/open-buildings-data/v1/polygons_s2_level_4_gzip/1a1_buildings.csv.gz',
       'https://storage.googleapis.com/open-buildings-data/v1/polygons_s2_level_4_gzip/1a3_buildings.csv.gz'],
      dtype=object)

In [8]:
b = selD.total_bounds
try:
    del final
except:
    pass
for tile in sel_tiles:
    cur_tile = os.path.join(buildings_folder, os.path.basename(tile)[:-3])
    curT = pd.read_csv(cur_tile)
    selT = curT.loc[curT['latitude'].between(b[1], b[3])]
    selT = selT.loc[selT['longitude'].between(b[0], b[2])]    
    try:
        final = final.append(selT)
    except:
        final = selT
    tPrint(f'{os.path.basename(tile)[:-3]}: {selT.shape}')
tPrint(f'Total buildings: {final.shape}')

11:13:25	199_buildings.csv: (579652, 6)
11:13:31	19f_buildings.csv: (19900, 6)
11:13:36	1a1_buildings.csv: (123620, 6)
11:13:46	1a3_buildings.csv: (1512487, 6)
11:13:46	Total buildings: (2235659, 6)


In [9]:
final.head()

,latitude,longitude,area_in_meters,confidence,geometry,full_plus_code
5,-6.340175,22.875338,6.2405,0.7248,"POLYGON((22.8753503473166 -6.34018559488285, 2...",6G54MV5G+W4P5
14,-6.342795,22.620962,21.7539,0.7769,"POLYGON((22.6209891242047 -6.34280690592974, 2...",6G54MJ4C+V9P5
18,-7.078167,23.038971,13.0139,0.6597,"POLYGON((23.0389883126813 -7.0781827953795, 23...",6G45W2CQ+PHQ6
23,-5.283744,23.227253,10.4586,0.7713,"POLYGON((23.227263859466 -5.28376259552161, 23...",6G65P68G+GW26
25,-5.598973,23.402233,20.3786,0.8575,"POLYGON((23.4022474009396 -5.59899805628671, 2...",6G65CC22+CV93


## Summarize Google Buildings

In [10]:
inB_geom = final['geometry'].apply(wkt.loads)
inB = gpd.GeoDataFrame(final, geometry=inB_geom, crs=4326)
inB.reset_index()

sindex = inB.sindex

In [11]:
selD['gBuildings'] = 0
for idx, row in selD.iterrows():
    potential_matches = sindex.intersection(row['geometry'].bounds)
    selB = inB.iloc[list(potential_matches)]
    exact_matches = selB.loc[selB.intersects(row['geometry'])]
    selD.loc[idx, 'gBuildings'] = exact_matches.shape[0]

In [12]:
selD.head()

,idx,Province,Health_Zone,IDP_2019,IDP_returned_2019,RepatRefugees,Nexus,Dwellings,share_idps,Selected,...,Pcode,cle,Sce_Sem,Modif,Code_DHIS2,Fiabilite,Code_SNIS,Anc_provin,geometry,gBuildings
0,29,Kasai central,Bena Tshiadi,18342,3958,No,No,22019,101.30%,1,...,CD9107ZS01,Bena TshiadiKasaï-Central,DSNIS,None,CXIGn9khu8A,None,None,Kasai-Occidental,"POLYGON ((22.91916 -4.52640, 22.94181 -4.52706...",21951
1,46,Kasai central,Bilomba,15259,58548,No,No,24334,303.30%,1,...,CD9105ZS01,BilombaKasaï-Central,DSNIS,None,FwiMpLozKls,None,None,Kasai-Occidental,"POLYGON ((22.25092 -6.37287, 22.26449 -6.38014...",23873
2,51,Kasai central,Bunkonde,13369,81923,No,Yes,25766,369.80%,1,...,CD9102ZS01,BunkondeKasaï-Central,DSNIS,None,Al39AWPqFCc,None,None,Kasai-Occidental,"POLYGON ((22.60655 -6.08197, 22.60759 -6.08208...",25748
3,41,Kasai oriental,Cilundu,31184,54810,No,No,35082,245.10%,1,...,CD8205ZS01,CilunduKasaï-Oriental,DSNIS,None,GZJ4wCF5dr8,None,None,Kasai-Oriental,"POLYGON ((23.51950 -6.40685, 23.51904 -6.40718...",37643
4,35,Kasai central,Demba,91215,40561,No,No,74879,176.00%,1,...,CD9106ZS02,DembaKasaï-Central,DSNIS,None,F8Spet7wqMK,None,None,Kasai-Occidental,"POLYGON ((22.41350 -5.09846, 22.41458 -5.09893...",72922


In [14]:
buildings_results = selD.loc[:,['idx','Province','Health_Zone','gBuildings','geometry']]
pd.DataFrame(buildings_results).to_csv(os.path.join(input_folder, "health_zone_building_counts.csv"))

,idx,Province,Health_Zone,gBuildings,geometry
0,29,Kasai central,Bena Tshiadi,21951,"POLYGON ((22.91916 -4.52640, 22.94181 -4.52706..."
1,46,Kasai central,Bilomba,23873,"POLYGON ((22.25092 -6.37287, 22.26449 -6.38014..."
2,51,Kasai central,Bunkonde,25748,"POLYGON ((22.60655 -6.08197, 22.60759 -6.08208..."
3,41,Kasai oriental,Cilundu,37643,"POLYGON ((23.51950 -6.40685, 23.51904 -6.40718..."
4,35,Kasai central,Demba,72922,"POLYGON ((22.41350 -5.09846, 22.41458 -5.09893..."


### Summarize OSM buildings

In [ ]:
osm_buildings_file = "/home/public/Data/COUNTRY/COD/OSM/GeoFabrik/gis_osm_buildings_a_free_1.shp"
osmB = gpd.read_file(osm_buildings_file)

osm_sindex = osmB.sindex

In [ ]:
selD['osmBuildings'] = 0
for idx, row in selD.iterrows():
    tPrint(idx)
    potential_matches = osm_sindex.intersection(row['geometry'].bounds)
    selB = osmB.iloc[list(potential_matches)]
    exact_matches = selB.loc[selB.intersects(row['geometry'])]
    selD.loc[idx, 'osmBuildings'] = exact_matches.shape[0]

In [ ]:
selD.head()

### Summarize population by class

In [ ]:
# summarize population by urban class
if not os.path.exists(local_pop):
    inP = rasterio.open(global_pop)
    rMisc.clipRaster(inP, selD, local_pop, crop=False)
    
if not os.path.exists(local_urban):
    inU = rasterio.open(global_urban)
    rMisc.clipRaster(inU, selD, local_urban, crop=False)

inP = rasterio.open(local_pop)
inU = rasterio.open(local_urban)

In [ ]:
inU_data = inU.read()
inP_data = inP.read()
unq_vals = np.unique(inU_data)
unq_vals[1:]

In [ ]:
resD = selD.to_crs(inP.crs)

In [ ]:
for val in unq_vals[1:]:
    curP = (inU_data == val) * inP_data
    with rMisc.create_rasterio_inmemory(inP.profile, curP) as popR:
        res = rMisc.zonalStats(resD, popR, minVal=0)
        res = pd.DataFrame(res, columns=['SUM', 'MIN', 'MAX', 'MEAN'])
        resD[f'Pop_{val}'] = res['SUM']    

In [ ]:
resD = resD.loc[:,['idx','Province','Health_Zone','Pop_10','Pop_11','Pop_12','Pop_13','Pop_21','Pop_22','Pop_23','Pop_30']]

# Create mapping data

In [ ]:
finalD = resD
finalD['geometry'] = selD['geometry']
finalD['bldgs'] = buildings_results['gBuildings']
finalD['bldgsOSM'] = selD['osmBuildings']
finalD = gpd.GeoDataFrame(finalD, geometry='geometry', crs=4326)
finalD.to_file(os.path.join(input_folder, "health_zone_summary.geojson"), driver="GeoJSON")